<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/Python/Solutions/timediff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 64.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=fb435566580f0fbd81d2165036545ff6e221d65b1a6a19e01a4c4bdb14d2fa3b
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("datediff")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [ ]:
df = spark.read\
        .format('csv')\
        .option('sep',',')\
        .option('inferSchema',True)\
        .option('header',True)\
        .load('/content/ts_format.csv')

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag,round,when,sum,row_number,lit,concat,to_date

In [ ]:
window_spec = Window\
.partitionBy('UserID')\
.orderBy('Timestamp')

In [ ]:
df = df.withColumn('lagby1',lag(df.Timestamp).over(window_spec))

In [ ]:
df = df.withColumn('mindiff',round((df.Timestamp.cast('long')-df.lagby1.cast('long'))/60))

In [ ]:
df = df.withColumn('new_session',when(df.mindiff < 120,0)\
                   .otherwise(1))

In [ ]:
window_spec1 = Window\
.partitionBy('UserID')\
.orderBy('Timestamp')\
.rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [ ]:
df = df.withColumn('sumsess',sum(df.new_session).over(window_spec1))

In [ ]:
df = df.withColumn('const',lit('_s'))

In [ ]:
df = df.withColumn('usersession',concat(df.UserID,df.const,df.sumsess))

In [ ]:
cols_to_drop=['const','sumsess','lagby1','mindiff','new_session']
df = df.drop(*cols_to_drop)

In [ ]:
df.show()

+-------------------+------+-----------+
|          Timestamp|UserID|usersession|
+-------------------+------+-----------+
|2019-07-01 12:01:19|    u1|      u1_s1|
|2019-07-01 14:01:19|    u1|      u1_s2|
|2019-07-01 15:01:19|    u1|      u1_s2|
|2019-07-02 12:01:19|    u1|      u1_s3|
|2019-07-03 12:01:19|    u1|      u1_s4|
|2019-07-03 14:01:19|    u1|      u1_s5|
|2019-07-01 12:01:19|    u2|      u2_s1|
|2019-07-02 12:01:19|    u2|      u2_s2|
|2019-07-01 14:01:19|    u3|      u3_s1|
|2019-07-03 12:01:19|    u3|      u3_s2|
|2019-07-01 15:01:19|    u4|      u4_s1|
|2019-07-03 14:01:19|    u4|      u4_s2|
+-------------------+------+-----------+



In [ ]:
daily_df = df
daily_df = daily_df.withColumn('date',to_date(daily_df.Timestamp,'yyyy-MM-dd'))

In [ ]:
window_spec_dailt = Window\
.partitionBy('date')\
.orderBy('Timestamp')

In [ ]:
daily_df = daily_df.withColumn('lagby1',lag(daily_df.Timestamp).over(window_spec_dailt))

In [ ]:
daily_df = daily_df.withColumn('mindiff',round(((daily_df.Timestamp.cast('long') - daily_df.lagby1.cast('long'))/60),2))

In [ ]:
daily_df = daily_df.withColumn('newsession',when(daily_df.mindiff < 120,0)\
                               .otherwise(1))

In [ ]:
daily_df = daily_df.withColumn('dailysession',sum(daily_df.newsession).over(window_spec_dailt))

In [ ]:
daily_df.show()

+-------------------+------+-----------+----------+-------------------+-------+----------+------------+
|          Timestamp|UserID|usersession|      date|             lagby1|mindiff|newsession|dailysession|
+-------------------+------+-----------+----------+-------------------+-------+----------+------------+
|2019-07-01 12:01:19|    u1|      u1_s1|2019-07-01|               null|   null|         1|           1|
|2019-07-01 12:01:19|    u2|      u2_s1|2019-07-01|2019-07-01 12:01:19|    0.0|         0|           1|
|2019-07-01 14:01:19|    u1|      u1_s2|2019-07-01|2019-07-01 12:01:19|  120.0|         1|           2|
|2019-07-01 14:01:19|    u3|      u3_s1|2019-07-01|2019-07-01 14:01:19|    0.0|         0|           2|
|2019-07-01 15:01:19|    u1|      u1_s2|2019-07-01|2019-07-01 14:01:19|   60.0|         0|           2|
|2019-07-01 15:01:19|    u4|      u4_s1|2019-07-01|2019-07-01 15:01:19|    0.0|         0|           2|
|2019-07-02 12:01:19|    u1|      u1_s3|2019-07-02|             

In [ ]:
daily_df.createOrReplaceTempView('daily_df')

In [ ]:
daily_session_count = spark.sql("""select date,dailysession,count(*) from daily_df
                                    group by date,dailysession""")

In [ ]:
daily_session_count = daily_session_count.select('date','dailysession')

In [ ]:
daily_session_count.show()

+----------+------------+
|      date|dailysession|
+----------+------------+
|2019-07-01|           1|
|2019-07-01|           2|
|2019-07-02|           1|
|2019-07-03|           1|
|2019-07-03|           2|
+----------+------------+



In [ ]:
daily_session_total_count = spark.sql("""select date,max(dailysession) from daily_df
                                    group by date""")

In [ ]:
daily_session_total_count.show()

+----------+-----------------+
|      date|max(dailysession)|
+----------+-----------------+
|2019-07-01|                2|
|2019-07-02|                1|
|2019-07-03|                2|
+----------+-----------------+

